In [ ]:
!pip install datasets


In [ ]:
# Финальная версия обучения модели bert-Roman

import pandas as pd
from transformers import BertTokenizer, BertForQuestionAnswering
from transformers import AdamW
from torch.utils.data import DataLoader
from datasets import Dataset
import torch

# Загрузка данных из Excel
data = pd.read_excel('/home/questions.xlsx')
dataset = Dataset.from_pandas(data)

# Загрузка предварительно обученного BERT и токенизатора
model = BertForQuestionAnswering.from_pretrained('cointegrated/rubert-tiny2')
tokenizer = BertTokenizer.from_pretrained('cointegrated/rubert-tiny2')


# Предобработка данных
def preprocess_data(example):
    question = example['question']
    answer = example['answer']

    encoding = tokenizer(question, answer, return_tensors='pt', padding=True, truncation=True)
    labels = {
        'start_positions': 0,  # Начало ответа всегда в начале токена
        'end_positions': len(encoding['input_ids'][0]) - 1  # Конец ответа в конце токена
    }

    return {**encoding, **labels}

dataset = dataset.map(preprocess_data)

# Настройка DataLoader для обучения
train_loader = DataLoader(dataset, batch_size=1, shuffle=True)

# Настройка оптимизатора и функции потерь
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Обучение модели
model.train()
for epoch in range(3):
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = torch.tensor(batch['input_ids'])
        attention_mask = torch.tensor(batch['attention_mask'])
        start_positions = torch.tensor(batch['start_positions'])
        end_positions = torch.tensor(batch['end_positions'])

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Сохранение обученной модели
model.save_pretrained("bert_qa_model")


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/269 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-17-1e7be5b3ceee>:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  start_positions = torch.tensor(batch['start_positions'])
<ipython-input-17-1e7be5b3ceee>:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  end_positions = torch.tensor(batch['end_positions'])


In [ ]:
# Сохранение обученной модели
model.save_pretrained("/home/bert_qa_model")